# **Промежуточная аттестация по курсу (Python)**

## **ЗАДАНИЕ**

В ячейке ниже представлен код, генерирующий DataFrame, который состоит всего из 1 столбца. Задача состоит в том, чтобы перевести его в one hot вид. Нельзя использовать get_dummies.





---



In [55]:
# Инсталляция эмулятора терминала, который можно запустить в Colab
# (ttps://github.com/singhsidhukuldeep/Google-Colab-Shell):

!pip install google-colab-shell

  Preparing metadata (setup.py) ... done
  Created wheel for google-colab-shell: filename=google_colab_shell-0.2-py3-none-any.whl size=4107 sha256=5bc882e922591bb83c60091dd10e912be2b4b05d31432e82b3cbb66661a15f02
  Stored in directory: /root/.cache/pip/wheels/f9/36/65/95dd4599be065418a9fe1f482674c8e716ce540f3f484681d2
Successfully built google-colab-shell


In [56]:
from google_colab_shell import getshell

In [57]:
# Запуск эмулятора терминала (для работы с git и GitHub):

getshell()



---



In [58]:
import pandas as pd

import random

In [316]:
lst = ['robot'] * 10
lst += ['human'] * 10
random.shuffle(lst)
data = pd.DataFrame({'whoAmI':lst})

data.head()

,whoAmI
0,robot
1,human
2,robot
3,human
4,human




---



## **РЕШЕНИЕ:**

### ВАРИАНТ

In [317]:
one_hot_df_1 = data

one_hot_df_1.head()

,whoAmI
0,robot
1,human
2,robot
3,human
4,human


In [318]:
# Добавляем новые поля 'robot' и 'human' и заполняем их соотвестствующими
# значениями в зависимости от содержания поля 'whoAmI':

one_hot_df_1.loc[one_hot_df_1['whoAmI'] == 'human', 'human'] = 1
one_hot_df_1.loc[one_hot_df_1['whoAmI'] == 'robot', 'robot'] = 1

one_hot_df_1.head()

,whoAmI,human,robot
0,robot,NaN,1.0
1,human,1.0,NaN
2,robot,NaN,1.0
3,human,1.0,NaN
4,human,1.0,NaN


In [319]:
# Заполняем пустые значения нулями:

one_hot_df_1 = one_hot_df_1.fillna(0)

one_hot_df_1.head()

,whoAmI,human,robot
0,robot,0.0,1.0
1,human,1.0,0.0
2,robot,0.0,1.0
3,human,1.0,0.0
4,human,1.0,0.0


In [320]:
# Удаляем поле 'whoAmI', т.к. данные в нем стали избыточны:

one_hot_df_1.drop('whoAmI', axis = 1, inplace = True)

one_hot_df_1.head()

,human,robot
0,0.0,1.0
1,1.0,0.0
2,0.0,1.0
3,1.0,0.0
4,1.0,0.0


In [321]:
# Меняем типы данных для полей:

one_hot_df_1 = one_hot_df_1[['human', 'robot']].astype(int)

one_hot_df_1.head()

,human,robot
0,0,1
1,1,0
2,0,1
3,1,0
4,1,0




---



### ВАРИАНТ

In [292]:
# Производим "сбрасывание" индекса, чтобы впоследствии
# иметь возможность обращаться к полю с индексами:

one_hot_df_1 = data.reset_index()

one_hot_df_1.head()

,index,whoAmI
0,0,robot
1,1,robot
2,2,robot
3,3,robot
4,4,human


In [293]:
# Добавляем вспомогателный столбец для заполнения значений:

one_hot_df_1['code'] = 1

one_hot_df_1.head()

,index,whoAmI,code
0,0,robot,1
1,1,robot,1
2,2,robot,1
3,3,robot,1
4,4,human,1


In [294]:
# Производим "сведение" датафрейма при помощи метода .pivot_table()
# (при этом отстутствующие значения заполняются нулями):

one_hot_df_1 = one_hot_df_1.pivot_table(index = 'index', columns = 'whoAmI', \
                                        values = 'code', fill_value = 0)

one_hot_df_1.head()

whoAmI,human,robot
index,,
0,0,1
1,0,1
2,0,1
3,0,1
4,1,0


In [295]:
# Убираем наименование индекса, который нумерует строки:
one_hot_df_1.rename_axis('', axis = 'index', inplace = True)

one_hot_df_1.head()

whoAmI,human,robot
,,
0,0,1
1,0,1
2,0,1
3,0,1
4,1,0


In [296]:
# Убираем наименование индекса, который находится в столбцах:
one_hot_df_1.rename_axis('', axis = 'columns', inplace = True)

one_hot_df_1.head()

,human,robot
,,
0,0,1
1,0,1
2,0,1
3,0,1
4,1,0




---



### ВАРИАНТ

In [202]:
# Трансфоммируем единственное поле с данными в ещё одно поле индекса:

one_hot_df = data.set_index(keys = 'whoAmI', append = True)

one_hot_df.head()

,
,whoAmI
0,human
1,robot
2,human
3,human
4,robot


In [203]:
# Добавляем вспомогателный столбец для заполнения значений:

one_hot_df['code'] = 1

# Убираем наименование индекса:
one_hot_df.index.rename(['', ''], inplace = True)

one_hot_df.head()

,,code
,,
0,human,1
1,robot,1
2,human,1
3,human,1
4,robot,1


In [204]:
# Производим "сворачивание" датафрейма при помощи метода .unstack()
# (при этом отстутствующие значения заполняются нулями):

one_hot_df = one_hot_df.unstack(fill_value = 0)

one_hot_df.head()

code      
  human robot
             
0     1     0
1     0     1
2     1     0
3     1     0
4     0     1

In [205]:
# Убираем верхний уровень в "шапке" датафрейма
# (т.е. удаляем уровень из мультииндекса):

one_hot_df.columns = one_hot_df.columns.droplevel()

one_hot_df.head()

,human,robot
,,
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1




---





---

